In [1]:
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver # driver de selenium
import time   # para manejar tiempo

In [2]:
from selenium.webdriver.chrome.options import Options   # opciones del driver
from selenium.webdriver.common.by import By   # para buscar segun tag, id, class, etc...
from selenium.webdriver.support.ui import WebDriverWait   # es para esperar
from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...
from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click
from selenium.webdriver.common.keys import Keys  # manejar teclas

In [3]:
# Además, hago la 'driver configuration'

opciones = Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

## Conecto con la página WEB

In [4]:
PATH = 'driver/chromedriver'

driver = webdriver.Chrome()   # abre una ventana de chrome 
                                # una vez que he abierto una ventana, puedo trabajar siempre sobre esta misma.

###   **1. Alternativa**

In [5]:
url = 'https://www.entradas.com/events/conciertos-y-festivales-85/alternativa-1791/'

driver.get(url)

In [6]:
div_shadow = driver.find_element(By.XPATH, '//div[@class="cmpwrapper"]')

In [7]:
# Check if the element has a Shadow DOM using JavaScript
has_shadow_dom = driver.execute_script('return arguments[0].shadowRoot !== null', div_shadow)

if has_shadow_dom:
    print("Element has a Shadow DOM")
else:
    print("Element does not have a Shadow DOM")

Element has a Shadow DOM


In [8]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', div_shadow)

In [9]:
shadow_root.find_elements(By.CSS_SELECTOR, '#cmpwelcomebtnyes > a')[0].click()

- Una vez consigo traspasar la barrera del 'Shadow DOM', entonces puedo comenzar con el scrapeo de las tablas que necesito para mi estudio. 

In [10]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [11]:
elementos = driver.find_elements(By.XPATH, '//*[@id="list-item-0"]')
elementos

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="6BD8908C8C5177D1EF812DE29B91EFEA_element_68")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="6BD8908C8C5177D1EF812DE29B91EFEA_element_69")>]

In [12]:
elementos[0].text.split('\n')


['Triángulo de Amor Bizarro',
 'Inverfest 2024',
 '29/12/20231 Evento',
 ' desde 19,80 €',
 'Mostrar 1 Evento']

In [13]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="6BD8908C8C5177D1EF812DE29B91EFEA_element_75")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="6BD8908C8C5177D1EF812DE29B91EFEA_element_49")>]

In [14]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//*[@class = "btn theme-button-radius btn-sm btn-block btn-primary theme-interaction-btn-bg theme-btn-font-color theme-btn-font-color-hover"]')
    enlaces = [enl.get_attribute('href') for enl in enlace]

    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Triángulo de Amor Bizarro', 'Cala Vento', 'Joe Crepúsculo', 'Alice Wonder', 'Momentos Alhambra: Rikas'], ['29/12/2023', '1 Evento', '01/12/2023', '1 Evento', '27/01/2024', '1 Evento', '17/11/2023', '1 Evento', '14/12/2023', '1 Evento'], ['desde 19,80 €', 'desde 19,80 €', 'desde 15,00 €', 'desde 27,00 €', 'desde 15,00 €'], ['https://www.entradas.com/artist/triangulo-de-amor-bizarro/', 'https://www.entradas.com/artist/cala-vento/', 'https://www.entradas.com/artist/joe-crepusculo/', 'https://www.entradas.com/artist/alice-wonder/', 'https://www.entradas.com/artist/momentos-alhambra/momentos-alhambra-rikas-3432438/']]
[['Triángulo de Amor Bizarro', 'Cala Vento', 'Joe Crepúsculo', 'Alice Wonder', 'Momentos Alhambra: Rikas'], ['29/12/2023', '1 Evento', '01/12/2023', '1 Evento', '27/01/2024', '1 Evento', '17/11/2023', '1 Evento', '14/12/2023', '1 Evento'], ['desde 19,80 €', 'desde 19,80 €', 'desde 15,00 €', 'desde 27,00 €', 'desde 15,00 €'], ['https://www.entradas.com/artist/triangulo-de-am

In [15]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [16]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['29/12/2023', '01/12/2023', '27/01/2024', '17/11/2023', '14/12/2023']
['1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento']


Importo los datos a un dataframe a través de pandas.

In [17]:
alternativa = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
alternativa

,artista,fechas,eventos,precios,enlaces
0,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
1,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
2,Joe Crepúsculo,27/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/
3,Alice Wonder,17/11/2023,1 Evento,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/
4,Momentos Alhambra: Rikas,14/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...


In [18]:
alternativa.insert(0, 'estilo', 'alternativa')
alternativa

,estilo,artista,fechas,eventos,precios,enlaces
0,alternativa,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
1,alternativa,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
2,alternativa,Joe Crepúsculo,27/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/
3,alternativa,Alice Wonder,17/11/2023,1 Evento,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/
4,alternativa,Momentos Alhambra: Rikas,14/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...


- Una vez que tengo los datos que quiero de la primera página de la web, repetimos el proceso para cada uno de los estilos de música. 

###   **2.Electrónica**

In [19]:
url = 'https://www.entradas.com/events/conciertos-y-festivales-85/electronica-1235/'

driver.get(url)

In [20]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [21]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="A7D1C271CCD54FD684D481FF4929242B_element_153")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="A7D1C271CCD54FD684D481FF4929242B_element_132")>]

In [22]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['OBK', 'Stee Downes/ Kyodai/ Ralph Session', 'Discotizer Presenta - Cody Currie + Javi Frias', 'Alain Perez & La Orquestra', 'MOOG', 'Parov Stelar - Theater Tour'], ['13/01/2024 ─ 27/01/2024', '3 eventos', '17/11/2023', '1 Evento', '18/11/2023', '1 Evento', '23/12/2023', '1 Evento', '12/11/2023 ─ 31/12/2023', '50 eventos', '25/04/2024 ─ 26/04/2024', '2 eventos'], ['desde 30,00 €', 'desde 15,00 €', 'desde 15,00 €', 'desde 22,00 €', 'desde 10,00 €', 'desde 40,00 €'], ['https://www.entradas.com/artist/obk/', 'https://www.entradas.com/artist/clubbing/stee-downes-kyodai-ralph-session-3525520/', 'https://www.entradas.com/artist/clubbing/discotizer-presenta-cody-currie-javi-frias-3518486/', 'https://www.entradas.com/artist/alain-perez-la-orquestra/', 'https://www.entradas.com/artist/moog/', 'https://www.entradas.com/artist/parov-stelar/']]
[['OBK', 'Stee Downes/ Kyodai/ Ralph Session', 'Discotizer Presenta - Cody Currie + Javi Frias', 'Alain Perez & La Orquestra', 'MOOG', 'Parov Stelar - Th

In [23]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [24]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['13/01/2024 ─ 27/01/2024', '17/11/2023', '18/11/2023', '23/12/2023', '12/11/2023 ─ 31/12/2023', '25/04/2024 ─ 26/04/2024']
['3 eventos', '1 Evento', '1 Evento', '1 Evento', '50 eventos', '2 eventos']


In [25]:
electronica = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
electronica

,artista,fechas,eventos,precios,enlaces
0,OBK,13/01/2024 ─ 27/01/2024,3 eventos,"desde 30,00 €",https://www.entradas.com/artist/obk/
1,Stee Downes/ Kyodai/ Ralph Session,17/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/clubbing/stee-...
2,Discotizer Presenta - Cody Currie + Javi Frias,18/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/clubbing/disco...
3,Alain Perez & La Orquestra,23/12/2023,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/alain-perez-la...
4,MOOG,12/11/2023 ─ 31/12/2023,50 eventos,"desde 10,00 €",https://www.entradas.com/artist/moog/
5,Parov Stelar - Theater Tour,25/04/2024 ─ 26/04/2024,2 eventos,"desde 40,00 €",https://www.entradas.com/artist/parov-stelar/


In [26]:
electronica.insert(0, 'estilo', 'electronica')
electronica

,estilo,artista,fechas,eventos,precios,enlaces
0,electronica,OBK,13/01/2024 ─ 27/01/2024,3 eventos,"desde 30,00 €",https://www.entradas.com/artist/obk/
1,electronica,Stee Downes/ Kyodai/ Ralph Session,17/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/clubbing/stee-...
2,electronica,Discotizer Presenta - Cody Currie + Javi Frias,18/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/clubbing/disco...
3,electronica,Alain Perez & La Orquestra,23/12/2023,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/alain-perez-la...
4,electronica,MOOG,12/11/2023 ─ 31/12/2023,50 eventos,"desde 10,00 €",https://www.entradas.com/artist/moog/
5,electronica,Parov Stelar - Theater Tour,25/04/2024 ─ 26/04/2024,2 eventos,"desde 40,00 €",https://www.entradas.com/artist/parov-stelar/


###    **3. Festivales**

In [27]:
url = 'https://www.entradas.com/events/conciertos-y-festivales-85/festivales-de-musica-1232/'

driver.get(url)

In [28]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [29]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="40F0EC23C5786E28EACBE177E2AB4BC7_element_255")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="40F0EC23C5786E28EACBE177E2AB4BC7_element_256")>]

In [30]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Iván Ferreiro', 'Warm Up Festival 2024', 'Spring Festival 2024', 'Low Festival 2024', 'Sex Museum', 'Gran Canaria SUM Festival 2024', 'GRANCA Live Fest 2024', 'Actual Festival 2024', 'Ebroclub 2024'], ['13/01/2024', '1 Evento', '03/05/2024 ─ 04/05/2024', '1 Evento', '31/05/2024 ─ 01/06/2024', '2 eventos', '26/07/2024 ─ 28/07/2024', '1 Evento', '06/01/2024', '1 Evento', '04/10/2024 ─ 05/10/2024', '3 eventos', '05/07/2024 ─ 06/07/2024', '1 Evento', '03/01/2024 ─ 04/01/2024', '1 Evento', '12/04/2024 ─ 13/04/2024', '1 Evento'], ['desde 38,50 €', 'desde 62,99 €', 'desde 41,99 €', 'desde 59,99 €', 'desde 22,00 €', 'desde 35,00 €', 'desde 89,00 €', 'desde 45,00 €', 'desde 35,00 €'], ['https://www.entradas.com/artist/ivan-ferreiro/ivan-ferreiro-3411430/', 'https://www.entradas.com/artist/warmupfestival/', 'https://www.entradas.com/artist/springfestival/', 'https://www.entradas.com/artist/low-festival/', 'https://www.entradas.com/artist/sex-museum/', 'https://www.entradas.com/artist/gran-can

In [31]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [32]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['13/01/2024', '03/05/2024 ─ 04/05/2024', '31/05/2024 ─ 01/06/2024', '26/07/2024 ─ 28/07/2024', '06/01/2024', '04/10/2024 ─ 05/10/2024', '05/07/2024 ─ 06/07/2024', '03/01/2024 ─ 04/01/2024', '12/04/2024 ─ 13/04/2024']
['1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '3 eventos', '1 Evento', '1 Evento', '1 Evento']


In [33]:
festivales = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
festivales

,artista,fechas,eventos,precios,enlaces
0,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/...
1,Warm Up Festival 2024,03/05/2024 ─ 04/05/2024,1 Evento,"desde 62,99 €",https://www.entradas.com/artist/warmupfestival/
2,Spring Festival 2024,31/05/2024 ─ 01/06/2024,2 eventos,"desde 41,99 €",https://www.entradas.com/artist/springfestival/
3,Low Festival 2024,26/07/2024 ─ 28/07/2024,1 Evento,"desde 59,99 €",https://www.entradas.com/artist/low-festival/
4,Sex Museum,06/01/2024,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/sex-museum/
5,Gran Canaria SUM Festival 2024,04/10/2024 ─ 05/10/2024,3 eventos,"desde 35,00 €",https://www.entradas.com/artist/gran-canaria-s...
6,GRANCA Live Fest 2024,05/07/2024 ─ 06/07/2024,1 Evento,"desde 89,00 €",https://www.entradas.com/artist/grancalivefest...
7,Actual Festival 2024,03/01/2024 ─ 04/01/2024,1 Evento,"desde 45,00 €",https://www.entradas.com/artist/festival-actual/
8,Ebroclub 2024,12/04/2024 ─ 13/04/2024,1 Evento,"desde 35,00 €",https://www.entradas.com/artist/ebroclub/


In [34]:
festivales.insert(0, 'estilo', 'festivales')
festivales

,estilo,artista,fechas,eventos,precios,enlaces
0,festivales,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/...
1,festivales,Warm Up Festival 2024,03/05/2024 ─ 04/05/2024,1 Evento,"desde 62,99 €",https://www.entradas.com/artist/warmupfestival/
2,festivales,Spring Festival 2024,31/05/2024 ─ 01/06/2024,2 eventos,"desde 41,99 €",https://www.entradas.com/artist/springfestival/
3,festivales,Low Festival 2024,26/07/2024 ─ 28/07/2024,1 Evento,"desde 59,99 €",https://www.entradas.com/artist/low-festival/
4,festivales,Sex Museum,06/01/2024,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/sex-museum/
5,festivales,Gran Canaria SUM Festival 2024,04/10/2024 ─ 05/10/2024,3 eventos,"desde 35,00 €",https://www.entradas.com/artist/gran-canaria-s...
6,festivales,GRANCA Live Fest 2024,05/07/2024 ─ 06/07/2024,1 Evento,"desde 89,00 €",https://www.entradas.com/artist/grancalivefest...
7,festivales,Actual Festival 2024,03/01/2024 ─ 04/01/2024,1 Evento,"desde 45,00 €",https://www.entradas.com/artist/festival-actual/
8,festivales,Ebroclub 2024,12/04/2024 ─ 13/04/2024,1 Evento,"desde 35,00 €",https://www.entradas.com/artist/ebroclub/


###    **4. Jazz y blues**

In [35]:
url = 'https://www.entradas.com/events/conciertos-y-festivales-85/jazz-blues-1236/'
driver.get(url)

In [36]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [37]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="B2A5E20CF2406890FF2252F73D338529_element_378")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="B2A5E20CF2406890FF2252F73D338529_element_333")>]

In [38]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['55 Festival de Jazz de Barcelona', 'Queralt Lahoz', 'John Grant', 'Antonio Lizana', 'Mark Guiliana', 'Billy Cobham Spectrum 50 Project', 'Shai Maestro', 'Scary Pockets', 'Calibro 35', 'Matthew Halsall', 'Fatoumata Diawara', 'Kassa Overall', 'Martirio & Chano Domínguez', 'Moisés P. Sánchez', "Iván 'Melon' Lewis & Ariel Brínguez", 'Aranlight Soul & Gospel - Jazz & the City', 'The Bamboos', 'Fila 8: Clásicos del Cine Moderno - Jazz & the City', 'Momentos Alhambra: Mammal Hands', 'Rai Paz Quartet'], ['13/11/2023 ─ 30/12/2023', '37 eventos', '01/12/2023 ─ 02/12/2023', '2 eventos', '17/11/2023', '1 Evento', '30/11/2023', '1 Evento', '20/11/2023', '1 Evento', '17/11/2023', '2 eventos', '12/11/2023', '1 Evento', '09/02/2024', '1 Evento', '24/11/2023', '1 Evento', '19/11/2023', '1 Evento', '21/11/2023', '1 Evento', '25/11/2023', '1 Evento', '14/11/2023', '1 Evento', '18/11/2023', '1 Evento', '16/12/2023', '2 eventos', '15/12/2023', '1 Evento', '16/11/2023', '1 Evento', '18/11/2023', '1 Event

In [39]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [40]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['13/11/2023 ─ 30/12/2023', '01/12/2023 ─ 02/12/2023', '17/11/2023', '30/11/2023', '20/11/2023', '17/11/2023', '12/11/2023', '09/02/2024', '24/11/2023', '19/11/2023', '21/11/2023', '25/11/2023', '14/11/2023', '18/11/2023', '16/12/2023', '15/12/2023', '16/11/2023', '18/11/2023', '18/11/2023', '15/12/2023']
['37 eventos', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '2 eventos']


In [41]:
jazz = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
jazz

,artista,fechas,eventos,precios,enlaces
0,55 Festival de Jazz de Barcelona,13/11/2023 ─ 30/12/2023,37 eventos,"desde 15,90 €",https://www.entradas.com/artist/voll-dammfesti...
1,Queralt Lahoz,01/12/2023 ─ 02/12/2023,2 eventos,"desde 23,00 €",https://www.entradas.com/artist/queralt-lahoz/
2,John Grant,17/11/2023,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/john-grant/
3,Antonio Lizana,30/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/villanos-del-j...
4,Mark Guiliana,20/11/2023,1 Evento,"desde 23,00 €",https://www.entradas.com/artist/mark-guiliana/
5,Billy Cobham Spectrum 50 Project,17/11/2023,2 eventos,"desde 29,00 €",https://www.entradas.com/artist/villanos-del-j...
6,Shai Maestro,12/11/2023,1 Evento,"desde 21,00 €",https://www.entradas.com/artist/villanos-del-j...
7,Scary Pockets,09/02/2024,1 Evento,"desde 28,00 €",https://www.entradas.com/artist/villanos-del-j...
8,Calibro 35,24/11/2023,1 Evento,"desde 21,00 €",https://www.entradas.com/artist/villanos-del-j...
9,Matthew Halsall,19/11/2023,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/villanos-del-j...


In [42]:
jazz.insert(0, 'estilo', 'jazz')
jazz

,estilo,artista,fechas,eventos,precios,enlaces
0,jazz,55 Festival de Jazz de Barcelona,13/11/2023 ─ 30/12/2023,37 eventos,"desde 15,90 €",https://www.entradas.com/artist/voll-dammfesti...
1,jazz,Queralt Lahoz,01/12/2023 ─ 02/12/2023,2 eventos,"desde 23,00 €",https://www.entradas.com/artist/queralt-lahoz/
2,jazz,John Grant,17/11/2023,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/john-grant/
3,jazz,Antonio Lizana,30/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/villanos-del-j...
4,jazz,Mark Guiliana,20/11/2023,1 Evento,"desde 23,00 €",https://www.entradas.com/artist/mark-guiliana/
5,jazz,Billy Cobham Spectrum 50 Project,17/11/2023,2 eventos,"desde 29,00 €",https://www.entradas.com/artist/villanos-del-j...
6,jazz,Shai Maestro,12/11/2023,1 Evento,"desde 21,00 €",https://www.entradas.com/artist/villanos-del-j...
7,jazz,Scary Pockets,09/02/2024,1 Evento,"desde 28,00 €",https://www.entradas.com/artist/villanos-del-j...
8,jazz,Calibro 35,24/11/2023,1 Evento,"desde 21,00 €",https://www.entradas.com/artist/villanos-del-j...
9,jazz,Matthew Halsall,19/11/2023,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/villanos-del-j...


#### Pasar de pagina (Jazz2)

In [43]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[5]/a/i')[0].click()

In [44]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [45]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3330367628F5C6E71DFF188821D6B0B6_element_591")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3330367628F5C6E71DFF188821D6B0B6_element_582")>]

In [46]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Yuhan Su Quartet - Liberated Gesture', 'Jemba Groove', 'JpBalcazar Quartet feat. Albert Bover', 'Rai Paz Quartet', 'Tornado Alley Jazz - De la noche una canción', 'Jordi Berni Trio', 'GAUR&NOALUR', 'Trovadores del Jazz', "The Poyo Moya Trio feat Pat O'Brien", 'Jamboree Jam', 'Invisible Tattoo', 'Marc Demuth & Irene Reig Quartet', 'Jocelyn Gould Quartet', 'Xavi Maldonado Quartet', 'Everybody needs some Christmas', 'Rhythm’N’Jamboree', 'Eva Català Group', 'Barcelona Big Blues Band feat Mitch Woods (USA)', 'Benjamin Herman', 'Blues & Roots'], ['24/11/2023', '2 eventos', '16/03/2024', '1 Evento', '19/12/2023', '2 eventos', '15/12/2023', '2 eventos', '24/11/2023', '1 Evento', '01/12/2023', '2 eventos', '19/11/2023', '1 Evento', '16/11/2023 ─ 27/12/2023', '8 eventos', '26/11/2023', '2 eventos', '13/11/2023 ─ 29/01/2024', '9 eventos', '29/11/2023', '1 Evento', '21/11/2023 ─ 22/11/2023', '2 eventos', '17/11/2023', '2 eventos', '16/01/2024', '2 eventos', '23/12/2023', '1 Evento', '03/12/2023

In [47]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [48]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['24/11/2023', '16/03/2024', '19/12/2023', '15/12/2023', '24/11/2023', '01/12/2023', '19/11/2023', '16/11/2023 ─ 27/12/2023', '26/11/2023', '13/11/2023 ─ 29/01/2024', '29/11/2023', '21/11/2023 ─ 22/11/2023', '17/11/2023', '16/01/2024', '23/12/2023', '03/12/2023 ─ 14/01/2024', '25/11/2023', '19/11/2023', '12/12/2023', '26/11/2023']
['2 eventos', '1 Evento', '2 eventos', '2 eventos', '1 Evento', '2 eventos', '1 Evento', '8 eventos', '2 eventos', '9 eventos', '1 Evento', '2 eventos', '2 eventos', '2 eventos', '1 Evento', '10 eventos', '1 Evento', '2 eventos', '2 eventos', '1 Evento']


In [49]:
jazz2 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
jazz2

,artista,fechas,eventos,precios,enlaces
0,Yuhan Su Quartet - Liberated Gesture,24/11/2023,2 eventos,"desde 18,00 €",https://www.entradas.com/artist/jamboree-jazz/...
1,Jemba Groove,16/03/2024,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/jemba-groove/
2,JpBalcazar Quartet feat. Albert Bover,19/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...
3,Rai Paz Quartet,15/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...
4,Tornado Alley Jazz - De la noche una canción,24/11/2023,1 Evento,"desde 10,00 €",https://www.entradas.com/artist/tornado-alley-...
5,Jordi Berni Trio,01/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...
6,GAUR&NOALUR,19/11/2023,1 Evento,"desde 4,25 €",https://www.entradas.com/artist/gaur-noalur/
7,Trovadores del Jazz,16/11/2023 ─ 27/12/2023,8 eventos,"desde 12,80 €",https://www.entradas.com/artist/trovadores-del...
8,The Poyo Moya Trio feat Pat O'Brien,26/11/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...
9,Jamboree Jam,13/11/2023 ─ 29/01/2024,9 eventos,"desde 12,00 €",https://www.entradas.com/artist/jamboree-jam/


In [50]:
jazz2.insert(0, 'estilo', 'jazz')
jazz2

,estilo,artista,fechas,eventos,precios,enlaces
0,jazz,Yuhan Su Quartet - Liberated Gesture,24/11/2023,2 eventos,"desde 18,00 €",https://www.entradas.com/artist/jamboree-jazz/...
1,jazz,Jemba Groove,16/03/2024,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/jemba-groove/
2,jazz,JpBalcazar Quartet feat. Albert Bover,19/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...
3,jazz,Rai Paz Quartet,15/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...
4,jazz,Tornado Alley Jazz - De la noche una canción,24/11/2023,1 Evento,"desde 10,00 €",https://www.entradas.com/artist/tornado-alley-...
5,jazz,Jordi Berni Trio,01/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...
6,jazz,GAUR&NOALUR,19/11/2023,1 Evento,"desde 4,25 €",https://www.entradas.com/artist/gaur-noalur/
7,jazz,Trovadores del Jazz,16/11/2023 ─ 27/12/2023,8 eventos,"desde 12,80 €",https://www.entradas.com/artist/trovadores-del...
8,jazz,The Poyo Moya Trio feat Pat O'Brien,26/11/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...
9,jazz,Jamboree Jam,13/11/2023 ─ 29/01/2024,9 eventos,"desde 12,00 €",https://www.entradas.com/artist/jamboree-jam/


#### Pasar de página (Jazz3)

In [51]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[5]/a/i')[0].click()

In [52]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [53]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3330367628F5C6E71DFF188821D6B0B6_element_591")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3330367628F5C6E71DFF188821D6B0B6_element_755")>]

In [54]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Blues & Roots', 'Emmet Cohen Trio', 'Irene Reig / The Bop Collective: Decisió i Coratge', 'Sweet Adu', 'Jon Urrutia Cuarteto + Jam session', 'Sant Andreu Reunion Band', 'Grant Stewart & Jim Rotondi West Coast', 'Jaume Vilaseca The Yes Octet', 'Gili-Romani Hot Jazz Cats', 'Smack Dab', 'Yoio Cuesta Quartet'], ['26/11/2023', '1 Evento', '13/11/2023 ─ 14/11/2023', '4 eventos', '18/11/2023', '2 eventos', '17/11/2023', '1 Evento', '01/12/2023', '1 Evento', '30/12/2023', '2 eventos', '25/11/2023', '2 eventos', '23/12/2023', '2 eventos', '29/12/2023', '2 eventos', '08/12/2023', '2 eventos', '14/12/2023', '1 Evento'], ['desde 9,00 €', 'desde 30,00 €', 'desde 18,00 €', 'desde 12,00 €', 'desde 6,91 €', 'desde 25,00 €', 'desde 35,00 €', 'desde 15,00 €', 'desde 28,00 €', 'desde 15,00 €', 'desde 12,00 €'], ['https://www.entradas.com/artist/blues-roots/', 'https://www.entradas.com/artist/emmet-cohen/', 'https://www.entradas.com/artist/irene-reig/', 'https://www.entradas.com/artist/sweet-adu/', 'ht

In [55]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [56]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['26/11/2023', '13/11/2023 ─ 14/11/2023', '18/11/2023', '17/11/2023', '01/12/2023', '30/12/2023', '25/11/2023', '23/12/2023', '29/12/2023', '08/12/2023', '14/12/2023']
['1 Evento', '4 eventos', '2 eventos', '1 Evento', '1 Evento', '2 eventos', '2 eventos', '2 eventos', '2 eventos', '2 eventos', '1 Evento']


In [57]:
jazz3 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
jazz3

,artista,fechas,eventos,precios,enlaces
0,Blues & Roots,26/11/2023,1 Evento,"desde 9,00 €",https://www.entradas.com/artist/blues-roots/
1,Emmet Cohen Trio,13/11/2023 ─ 14/11/2023,4 eventos,"desde 30,00 €",https://www.entradas.com/artist/emmet-cohen/
2,Irene Reig / The Bop Collective: Decisió i Cor...,18/11/2023,2 eventos,"desde 18,00 €",https://www.entradas.com/artist/irene-reig/
3,Sweet Adu,17/11/2023,1 Evento,"desde 12,00 €",https://www.entradas.com/artist/sweet-adu/
4,Jon Urrutia Cuarteto + Jam session,01/12/2023,1 Evento,"desde 6,91 €",https://www.entradas.com/artist/jon-urrutia-trio/
5,Sant Andreu Reunion Band,30/12/2023,2 eventos,"desde 25,00 €",https://www.entradas.com/artist/jamboree-jazz/...
6,Grant Stewart & Jim Rotondi West Coast,25/11/2023,2 eventos,"desde 35,00 €",https://www.entradas.com/artist/jamboree-jazz/...
7,Jaume Vilaseca The Yes Octet,23/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jaume-vilaseca...
8,Gili-Romani Hot Jazz Cats,29/12/2023,2 eventos,"desde 28,00 €",https://www.entradas.com/artist/jamboree-jazz/...
9,Smack Dab,08/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...


In [58]:
jazz3.insert(0, 'estilo', 'jazz')
jazz3

,estilo,artista,fechas,eventos,precios,enlaces
0,jazz,Blues & Roots,26/11/2023,1 Evento,"desde 9,00 €",https://www.entradas.com/artist/blues-roots/
1,jazz,Emmet Cohen Trio,13/11/2023 ─ 14/11/2023,4 eventos,"desde 30,00 €",https://www.entradas.com/artist/emmet-cohen/
2,jazz,Irene Reig / The Bop Collective: Decisió i Cor...,18/11/2023,2 eventos,"desde 18,00 €",https://www.entradas.com/artist/irene-reig/
3,jazz,Sweet Adu,17/11/2023,1 Evento,"desde 12,00 €",https://www.entradas.com/artist/sweet-adu/
4,jazz,Jon Urrutia Cuarteto + Jam session,01/12/2023,1 Evento,"desde 6,91 €",https://www.entradas.com/artist/jon-urrutia-trio/
5,jazz,Sant Andreu Reunion Band,30/12/2023,2 eventos,"desde 25,00 €",https://www.entradas.com/artist/jamboree-jazz/...
6,jazz,Grant Stewart & Jim Rotondi West Coast,25/11/2023,2 eventos,"desde 35,00 €",https://www.entradas.com/artist/jamboree-jazz/...
7,jazz,Jaume Vilaseca The Yes Octet,23/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jaume-vilaseca...
8,jazz,Gili-Romani Hot Jazz Cats,29/12/2023,2 eventos,"desde 28,00 €",https://www.entradas.com/artist/jamboree-jazz/...
9,jazz,Smack Dab,08/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/jamboree-jazz/...


###    **5.  Latino**

In [59]:
url = 'https://www.entradas.com/events/conciertos-y-festivales-85/latino-1792/'

driver.get(url)

In [60]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [61]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="FBCFA18785C36D822C22E13D53B173F3_element_851")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="FBCFA18785C36D822C22E13D53B173F3_element_852")>]

In [62]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Francisco', 'Vive La Salsa', 'Homenaje a Pablo Milanés', 'Rawayana - ¿Quien trae las cornetas?', 'Mariah Angeliq', 'Compota de Manana'], ['12/01/2024 ─ 23/03/2024', '4 eventos', '19/01/2024', '1 Evento', '24/03/2024', '1 Evento', '07/04/2024 ─ 10/04/2024', '3 eventos', '02/12/2023', '1 Evento', '03/12/2023', '1 Evento'], ['desde 30,00 €', 'desde 30,00 €', 'desde 30,00 €', 'desde 35,00 €', 'desde 27,29 €', 'desde 15,00 €'], ['https://www.entradas.com/artist/francisco/', 'https://www.entradas.com/artist/vive-la-salsa/', 'https://www.entradas.com/artist/homenaje-a-pablo-milanes/', 'https://www.entradas.com/artist/rawayana/', 'https://www.entradas.com/artist/mariah-angeliq/', 'https://www.entradas.com/artist/compota-de-manana/']]
[['Francisco', 'Vive La Salsa', 'Homenaje a Pablo Milanés', 'Rawayana - ¿Quien trae las cornetas?', 'Mariah Angeliq', 'Compota de Manana'], ['12/01/2024 ─ 23/03/2024', '4 eventos', '19/01/2024', '1 Evento', '24/03/2024', '1 Evento', '07/04/2024 ─ 10/04/2024', '

In [63]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [64]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['12/01/2024 ─ 23/03/2024', '19/01/2024', '24/03/2024', '07/04/2024 ─ 10/04/2024', '02/12/2023', '03/12/2023']
['4 eventos', '1 Evento', '1 Evento', '3 eventos', '1 Evento', '1 Evento']


In [65]:
latino = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
latino

,artista,fechas,eventos,precios,enlaces
0,Francisco,12/01/2024 ─ 23/03/2024,4 eventos,"desde 30,00 €",https://www.entradas.com/artist/francisco/
1,Vive La Salsa,19/01/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/vive-la-salsa/
2,Homenaje a Pablo Milanés,24/03/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/homenaje-a-pab...
3,Rawayana - ¿Quien trae las cornetas?,07/04/2024 ─ 10/04/2024,3 eventos,"desde 35,00 €",https://www.entradas.com/artist/rawayana/
4,Mariah Angeliq,02/12/2023,1 Evento,"desde 27,29 €",https://www.entradas.com/artist/mariah-angeliq/
5,Compota de Manana,03/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/compota-de-man...


In [66]:
latino.insert(0, 'estilo', 'latino')
latino

,estilo,artista,fechas,eventos,precios,enlaces
0,latino,Francisco,12/01/2024 ─ 23/03/2024,4 eventos,"desde 30,00 €",https://www.entradas.com/artist/francisco/
1,latino,Vive La Salsa,19/01/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/vive-la-salsa/
2,latino,Homenaje a Pablo Milanés,24/03/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/homenaje-a-pab...
3,latino,Rawayana - ¿Quien trae las cornetas?,07/04/2024 ─ 10/04/2024,3 eventos,"desde 35,00 €",https://www.entradas.com/artist/rawayana/
4,latino,Mariah Angeliq,02/12/2023,1 Evento,"desde 27,29 €",https://www.entradas.com/artist/mariah-angeliq/
5,latino,Compota de Manana,03/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/compota-de-man...


###    **6.  Clasica**

In [67]:
url = 'https://www.entradas.com/events/conciertos-y-festivales-85/musica-clasica-1237/'

driver.get(url)

In [68]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [69]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3A071A3F94FF782A7FF88A2EEE8E317E_element_961")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3A071A3F94FF782A7FF88A2EEE8E317E_element_921")>]

In [70]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['The World Of Hans Zimmer', 'Ara Malikian', 'Austin TV', 'Concierto Sinfónico de Abono A13 (Jueves)', 'Concierto Sinfónico de Abono B18 (Jueves)', 'Concierto Sinfónico de Abono B16 (Viernes)', 'Concierto Sinfónico de Abono B12 (Viernes)', 'Concierto Sinfónico de Abono A7 (Viernes)', 'Concierto Sinfónico de Abono B16 (Jueves)', 'Concierto Sinfónico de Abono B18 (Viernes)', 'Concierto Sinfónico de Abono A15 (Jueves)', 'Concierto Sinfónico de Abono A19 (Viernes)', 'Concierto Sinfónico de Abono A15 (Viernes)', 'Concierto Sinfónico de Abono A7 (Jueves)', 'Concierto Sinfónico de Abono B8 (Viernes)', 'Concierto Sinfónico de Abono A11 (Viernes)', 'Concierto Sinfónico de Abono A17 (Viernes)', 'Concierto Sinfónico de Abono A13 (Viernes)', 'Concierto Sinfónico de Abono A9 (Jueves)', 'Concierto Sinfónico de Abono B14 (Jueves)'], ['24/04/2024 ─ 29/04/2024', '2 eventos', '11/02/2024', '1 Evento', '31/01/2024', '1 Evento', '22/02/2024', '1 Evento', '18/04/2024', '1 Evento', '22/03/2024', '1 Evento'

In [71]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [72]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['24/04/2024 ─ 29/04/2024', '11/02/2024', '31/01/2024', '22/02/2024', '18/04/2024', '22/03/2024', '16/02/2024', '24/11/2023', '21/03/2024', '19/04/2024', '14/03/2024', '26/04/2024', '15/03/2024', '23/11/2023', '12/01/2024', '09/02/2024', '12/04/2024', '23/02/2024', '18/01/2024', '29/02/2024']
['2 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento']


In [73]:
clasica = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
clasica

,artista,fechas,eventos,precios,enlaces
0,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
1,Ara Malikian,11/02/2024,1 Evento,"desde 55,00 €",https://www.entradas.com/artist/ara-malikian/a...
2,Austin TV,31/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/inverfest/aust...
3,Concierto Sinfónico de Abono A13 (Jueves),22/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
4,Concierto Sinfónico de Abono B18 (Jueves),18/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
5,Concierto Sinfónico de Abono B16 (Viernes),22/03/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
6,Concierto Sinfónico de Abono B12 (Viernes),16/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
7,Concierto Sinfónico de Abono A7 (Viernes),24/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
8,Concierto Sinfónico de Abono B16 (Jueves),21/03/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
9,Concierto Sinfónico de Abono B18 (Viernes),19/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...


In [74]:
clasica.insert(0, 'estilo', 'clasica')
clasica

,estilo,artista,fechas,eventos,precios,enlaces
0,clasica,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
1,clasica,Ara Malikian,11/02/2024,1 Evento,"desde 55,00 €",https://www.entradas.com/artist/ara-malikian/a...
2,clasica,Austin TV,31/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/inverfest/aust...
3,clasica,Concierto Sinfónico de Abono A13 (Jueves),22/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
4,clasica,Concierto Sinfónico de Abono B18 (Jueves),18/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
5,clasica,Concierto Sinfónico de Abono B16 (Viernes),22/03/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
6,clasica,Concierto Sinfónico de Abono B12 (Viernes),16/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
7,clasica,Concierto Sinfónico de Abono A7 (Viernes),24/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
8,clasica,Concierto Sinfónico de Abono B16 (Jueves),21/03/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
9,clasica,Concierto Sinfónico de Abono B18 (Viernes),19/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...


#### Pasar de página (Clasica2)

In [75]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[6]/a/i')[0].click()

In [76]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [77]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3A071A3F94FF782A7FF88A2EEE8E317E_element_961")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3A071A3F94FF782A7FF88A2EEE8E317E_element_1115")>]

In [78]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Concierto Sinfónico de Abono A9 (Jueves)', 'Concierto Sinfónico de Abono B8 (Jueves)', 'Concierto Sinfónico de Abono B20 (Jueves)', 'JÓVENES MÚSICOS II', 'Concierto Sinfónico de Abono A9 (Viernes)', 'Concierto Sinfónico de Abono A11 (Jueves)', 'CONCIERTO DE GALARDONADOS EL CONCURSO JUVENTUDES MUSICALES', 'Concierto Sinfónico de Abono B12 (Jueves)', 'JÓVENES MÚSICOS III', 'Concierto Sinfónico de Abono A19 (Jueves)', 'Concierto Sinfónico de Abono B10 (Jueves)', 'Ensayos Ciclo Sinfónico', 'Concierto Sinfónico de Abono A17 (Jueves)', 'Concierto Sinfónico de Abono B14 (Viernes)', 'JÓVENES MÚSICOS I', 'Concierto Sinfónico de Abono B20 (Viernes)', 'Andrea Bocelli', 'Tina Turner Sinfónico', 'Elvis in my life', '“Skybridges”'], ['18/01/2024', '1 Evento', '11/01/2024', '1 Evento', '09/05/2024', '1 Evento', '02/02/2024', '1 Evento', '19/01/2024', '1 Evento', '08/02/2024', '1 Evento', '17/05/2024', '1 Evento', '15/02/2024', '1 Evento', '05/04/2024', '1 Evento', '25/04/2024', '1 Evento', '25/01/

In [79]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [80]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['18/01/2024', '11/01/2024', '09/05/2024', '02/02/2024', '19/01/2024', '08/02/2024', '17/05/2024', '15/02/2024', '05/04/2024', '25/04/2024', '25/01/2024', '23/11/2023 ─ 09/05/2024', '11/04/2024', '01/03/2024', '17/11/2023', '10/05/2024', '01/05/2024', '24/05/2024 ─ 25/05/2024', '26/01/2024', '08/03/2024']
['1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '14 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento']


In [81]:
clasica2 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
clasica2

,artista,fechas,eventos,precios,enlaces
0,Concierto Sinfónico de Abono A9 (Jueves),18/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
1,Concierto Sinfónico de Abono B8 (Jueves),11/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
2,Concierto Sinfónico de Abono B20 (Jueves),09/05/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
3,JÓVENES MÚSICOS II,02/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
4,Concierto Sinfónico de Abono A9 (Viernes),19/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
5,Concierto Sinfónico de Abono A11 (Jueves),08/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
6,CONCIERTO DE GALARDONADOS EL CONCURSO JUVENTUD...,17/05/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
7,Concierto Sinfónico de Abono B12 (Jueves),15/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
8,JÓVENES MÚSICOS III,05/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
9,Concierto Sinfónico de Abono A19 (Jueves),25/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...


In [82]:
clasica2.insert(0, 'estilo', 'clasica')
clasica2

,estilo,artista,fechas,eventos,precios,enlaces
0,clasica,Concierto Sinfónico de Abono A9 (Jueves),18/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
1,clasica,Concierto Sinfónico de Abono B8 (Jueves),11/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
2,clasica,Concierto Sinfónico de Abono B20 (Jueves),09/05/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
3,clasica,JÓVENES MÚSICOS II,02/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
4,clasica,Concierto Sinfónico de Abono A9 (Viernes),19/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
5,clasica,Concierto Sinfónico de Abono A11 (Jueves),08/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
6,clasica,CONCIERTO DE GALARDONADOS EL CONCURSO JUVENTUD...,17/05/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
7,clasica,Concierto Sinfónico de Abono B12 (Jueves),15/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
8,clasica,JÓVENES MÚSICOS III,05/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
9,clasica,Concierto Sinfónico de Abono A19 (Jueves),25/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...


#### Pasar de pagina (clasica3)

In [83]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[6]/a/i')[0].click()

In [84]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [85]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3A071A3F94FF782A7FF88A2EEE8E317E_element_961")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3A071A3F94FF782A7FF88A2EEE8E317E_element_1274")>]

In [86]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Zree and more', 'Concierto Conmemorativo 50 Aniversario', '“De aquí”', '“Dedication II”', 'Walking Jazz "AtriLogy"', 'Joni Jiménez', 'Lara Bello flamenco', 'Antonio Reyes (flamenco)', 'Concierto de Navidad 2023', 'Celtiberia Jazz', 'Bloomsound', 'Lo sugerente', '"Suite Sefarad"', 'Rajiv Jayawer', 'Ocaña, Copla de Libertad', 'Niño Josele', 'Jazz Mad Bones: Tributo a Glenn Miller', '"Recital 2.0"', '“Ritual live”', 'Concierto Extraordinario'], ['15/03/2024', '1 Evento', '01/12/2023', '1 Evento', '02/02/2024', '1 Evento', '16/02/2024', '1 Evento', '22/03/2024', '1 Evento', '31/05/2024', '1 Evento', '12/04/2024', '1 Evento', '01/12/2023', '1 Evento', '21/12/2023 ─ 22/12/2023', '2 eventos', '19/04/2024', '1 Evento', '24/05/2024', '1 Evento', '05/04/2024', '1 Evento', '24/11/2023', '1 Evento', '26/04/2024', '1 Evento', '17/11/2023', '1 Evento', '01/03/2024', '1 Evento', '12/01/2024', '1 Evento', '15/12/2023', '1 Evento', '23/02/2024', '1 Evento', '14/12/2023', '1 Evento'], ['desde 15,00 €

In [87]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [88]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['15/03/2024', '01/12/2023', '02/02/2024', '16/02/2024', '22/03/2024', '31/05/2024', '12/04/2024', '01/12/2023', '21/12/2023 ─ 22/12/2023', '19/04/2024', '24/05/2024', '05/04/2024', '24/11/2023', '26/04/2024', '17/11/2023', '01/03/2024', '12/01/2024', '15/12/2023', '23/02/2024', '14/12/2023']
['1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento']


In [89]:
clasica3 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
clasica3

,artista,fechas,eventos,precios,enlaces
0,Zree and more,15/03/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
1,Concierto Conmemorativo 50 Aniversario,01/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
2,“De aquí”,02/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
3,“Dedication II”,16/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
4,"Walking Jazz ""AtriLogy""",22/03/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
5,Joni Jiménez,31/05/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
6,Lara Bello flamenco,12/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
7,Antonio Reyes (flamenco),01/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
8,Concierto de Navidad 2023,21/12/2023 ─ 22/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
9,Celtiberia Jazz,19/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...


In [90]:
clasica3.insert(0, 'estilo', 'clasica')
clasica3

,estilo,artista,fechas,eventos,precios,enlaces
0,clasica,Zree and more,15/03/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
1,clasica,Concierto Conmemorativo 50 Aniversario,01/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
2,clasica,“De aquí”,02/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
3,clasica,“Dedication II”,16/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
4,clasica,"Walking Jazz ""AtriLogy""",22/03/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
5,clasica,Joni Jiménez,31/05/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
6,clasica,Lara Bello flamenco,12/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
7,clasica,Antonio Reyes (flamenco),01/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
8,clasica,Concierto de Navidad 2023,21/12/2023 ─ 22/12/2023,2 eventos,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
9,clasica,Celtiberia Jazz,19/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...


#### Pasar de pagina (clasica4)

In [91]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[6]/a/i')[0].click()

In [92]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [93]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3A071A3F94FF782A7FF88A2EEE8E317E_element_961")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496206ad01ffd9e8d71c8a6e8de8a1c1", element="3A071A3F94FF782A7FF88A2EEE8E317E_element_1413")>]

In [94]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Rajiv Jayawer', 'Ocaña, Copla de Libertad', '"Suite Sefarad"', 'Jazz Mad Bones: Tributo a Glenn Miller', 'Ludovico Musical Experience', 'Joven Orquesta Nacional de España', 'Orquesta de Cámara Franz Liszt', 'Homenaje a John Williams y lo mejor de Disney', 'Ministriles Hispalensis y Cynthia Luque', 'Kenneth Weiss', 'Concierto de año nuevo - Repertorio clásico de Viena', 'Música de películas', 'Christmas Concert', 'Noche de Jóvenes Estrellas', 'Federico Albanese', 'Zubin Mehta y Orquesta Filarmónica de Múnich', 'Luís González', 'El Conciertazo en Familia: Especial Navidad', 'El Conciertazo en Familia: Mozart y sus amigos', 'Johann Strauss Gran Concierto de Año Nuevo. Strauss Festival Orchestra con ballet', 'El Conciertazo en Familia: Las Cuatro Estaciones', 'Schumann y Brahms - Ensemble Praeteritum', 'Concierto de Música Clásica', 'La Serva Padrona - Aranjuez', 'Mujer, inspiración y acción', 'Los Tres Tenores', 'Evgeni Kissin', 'Ramon Humet & Sílvia Vidal', 'La Novena de Beethoven y O

In [95]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [96]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['26/04/2024', '17/11/2023', '24/11/2023', '12/01/2024', '22/12/2023', '12/01/2024', '26/11/2023', '23/12/2023', '12/11/2023', '19/11/2023', '01/01/2024 ─ 02/01/2024', '13/01/2024', '22/12/2023', '16/11/2023', '23/11/2023', '28/01/2024', '02/03/2024', '30/12/2023', '25/05/2024', '20/01/2024', '16/03/2024', '16/11/2023', '02/12/2023', '25/11/2023', '19/11/2023', '22/12/2023', '08/01/2024', '21/03/2024', '13/12/2023', '16/11/2023', '07/03/2024', '18/12/2023', '03/12/2023', '19/11/2023', '17/12/2023', '29/04/2024', '16/02/2024', '27/05/2024', '01/01/2024']
['1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '5 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Eve

In [97]:
clasica4 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
clasica4

,artista,fechas,eventos,precios,enlaces
0,Rajiv Jayawer,26/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
1,"Ocaña, Copla de Libertad",17/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
2,"""Suite Sefarad""",24/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
3,Jazz Mad Bones: Tributo a Glenn Miller,12/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
4,Ludovico Musical Experience,22/12/2023,1 Evento,"desde 17,60 €",https://www.entradas.com/artist/ludovico-music...
5,Joven Orquesta Nacional de España,12/01/2024,1 Evento,"desde 16,00 €",https://www.entradas.com/artist/riojaforum-pro...
6,Orquesta de Cámara Franz Liszt,26/11/2023,1 Evento,"desde 29,00 €",https://www.entradas.com/artist/riojaforum-pro...
7,Homenaje a John Williams y lo mejor de Disney,23/12/2023,1 Evento,"desde 38,00 €",https://www.entradas.com/artist/royal-film-con...
8,Ministriles Hispalensis y Cynthia Luque,12/11/2023,1 Evento,Actualmente no disponible,https://www.entradas.com/artist/festival-de-mu...
9,Kenneth Weiss,19/11/2023,1 Evento,"desde 16,25 €",https://www.entradas.com/artist/festival-de-mu...


In [98]:
clasica4.insert(0, 'estilo', 'clasica')
clasica4

,estilo,artista,fechas,eventos,precios,enlaces
0,clasica,Rajiv Jayawer,26/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
1,clasica,"Ocaña, Copla de Libertad",17/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
2,clasica,"""Suite Sefarad""",24/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
3,clasica,Jazz Mad Bones: Tributo a Glenn Miller,12/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/eventos-del-te...
4,clasica,Ludovico Musical Experience,22/12/2023,1 Evento,"desde 17,60 €",https://www.entradas.com/artist/ludovico-music...
5,clasica,Joven Orquesta Nacional de España,12/01/2024,1 Evento,"desde 16,00 €",https://www.entradas.com/artist/riojaforum-pro...
6,clasica,Orquesta de Cámara Franz Liszt,26/11/2023,1 Evento,"desde 29,00 €",https://www.entradas.com/artist/riojaforum-pro...
7,clasica,Homenaje a John Williams y lo mejor de Disney,23/12/2023,1 Evento,"desde 38,00 €",https://www.entradas.com/artist/royal-film-con...
8,clasica,Ministriles Hispalensis y Cynthia Luque,12/11/2023,1 Evento,Actualmente no disponible,https://www.entradas.com/artist/festival-de-mu...
9,clasica,Kenneth Weiss,19/11/2023,1 Evento,"desde 16,25 €",https://www.entradas.com/artist/festival-de-mu...


## Unión de Dataframes

In [99]:
events = pd.concat([alternativa, electronica, festivales, jazz, jazz2, jazz3, latino, clasica, clasica2, clasica3, clasica4])

,estilo,artista,fechas,eventos,precios,enlaces
34,clasica,Gala de Zarzuela en Navidad,17/12/2023,1 Evento,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...
35,clasica,Ton Koopman y Orquesta Barroca de Ámsterdam,29/04/2024,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/ton-koopman-i-...
36,clasica,Hilary Hahn y Gianandrea Noseda,16/02/2024,1 Evento,"desde 32,00 €",https://www.entradas.com/artist/hilary-hahn-i-...
37,clasica,Hélène Grimaud,27/05/2024,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/helene-grimaud/
38,clasica,Concierto Año Nuevo - The Orbist Internacional...,01/01/2024,1 Evento,"desde 35,00 €",https://www.entradas.com/artist/the-orbist-int...


In [100]:
events.reset_index(inplace=True)

In [101]:
events.tail()

,index,estilo,artista,fechas,eventos,precios,enlaces
171,34,clasica,Gala de Zarzuela en Navidad,17/12/2023,1 Evento,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...
172,35,clasica,Ton Koopman y Orquesta Barroca de Ámsterdam,29/04/2024,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/ton-koopman-i-...
173,36,clasica,Hilary Hahn y Gianandrea Noseda,16/02/2024,1 Evento,"desde 32,00 €",https://www.entradas.com/artist/hilary-hahn-i-...
174,37,clasica,Hélène Grimaud,27/05/2024,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/helene-grimaud/
175,38,clasica,Concierto Año Nuevo - The Orbist Internacional...,01/01/2024,1 Evento,"desde 35,00 €",https://www.entradas.com/artist/the-orbist-int...


In [102]:
events.drop(columns="index",inplace=True)

In [103]:
events.head(20)

,estilo,artista,fechas,eventos,precios,enlaces
0,alternativa,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
1,alternativa,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
2,alternativa,Joe Crepúsculo,27/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/
3,alternativa,Alice Wonder,17/11/2023,1 Evento,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/
4,alternativa,Momentos Alhambra: Rikas,14/12/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...
5,electronica,OBK,13/01/2024 ─ 27/01/2024,3 eventos,"desde 30,00 €",https://www.entradas.com/artist/obk/
6,electronica,Stee Downes/ Kyodai/ Ralph Session,17/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/clubbing/stee-...
7,electronica,Discotizer Presenta - Cody Currie + Javi Frias,18/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/clubbing/disco...
8,electronica,Alain Perez & La Orquestra,23/12/2023,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/alain-perez-la...
9,electronica,MOOG,12/11/2023 ─ 31/12/2023,50 eventos,"desde 10,00 €",https://www.entradas.com/artist/moog/


In [104]:
events.tail(20)

,estilo,artista,fechas,eventos,precios,enlaces
156,clasica,Johann Strauss Gran Concierto de Año Nuevo. St...,20/01/2024,1 Evento,"desde 41,00 €",https://www.entradas.com/artist/strauss-festiv...
157,clasica,El Conciertazo en Familia: Las Cuatro Estaciones,16/03/2024,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/el-conciertazo...
158,clasica,Schumann y Brahms - Ensemble Praeteritum,16/11/2023,1 Evento,"desde 12,00 €",https://www.entradas.com/artist/ensemble-praet...
159,clasica,Concierto de Música Clásica,02/12/2023,1 Evento,"desde 13,00 €",https://www.entradas.com/artist/concierto-de-m...
160,clasica,La Serva Padrona - Aranjuez,25/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/la-serva-padro...
161,clasica,"Mujer, inspiración y acción",19/11/2023,1 Evento,"desde 4,17 €",https://www.entradas.com/artist/mujer-inspirac...
162,clasica,Los Tres Tenores,22/12/2023,1 Evento,"desde 12,00 €",https://www.entradas.com/artist/los-tres-tenores/
163,clasica,Evgeni Kissin,08/01/2024,1 Evento,"desde 25,00 €",https://www.entradas.com/artist/evgeni-kissin/
164,clasica,Ramon Humet & Sílvia Vidal,21/03/2024,1 Evento,"desde 25,00 €",https://www.entradas.com/artist/ramon-humet/
165,clasica,La Novena de Beethoven y Orfeón Donostiarra,13/12/2023,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/novena-de-beet...


Ahora que tengo el dataframe final voy a terminar de limpiarlo. 

- Separo las fechas en 2 columnas, ya que hay muchos eventos que tienen más de una fecha.
- En aquellas filas que solamente tengan una fecha, pido que se deje un espacio vacío en la columna 'fecha2'

In [105]:
events = pd.DataFrame(events)

# Separa las fechas en fecha1 y fecha2

events['fecha1'] = events['fechas'].str.split(' ─ ').str[0]
events['fecha2'] = events['fechas'].str.split(' ─ ').str[1]

# Elimina espacios en blanco adicionales en las fechas

events['fecha1'] = events['fecha1'].str.strip()
events['fecha2'] = events['fecha2'].str.strip()

# Rellena las fechas faltantes en fecha2 con NaN

events['fecha2'].fillna('', inplace=True)

# Elimina la columna original 'fechas' si es necesario

events.drop('fechas', axis=1, inplace=True)

In [108]:
events.head()

,estilo,artista,eventos,precios,enlaces,fecha1,fecha2
0,alternativa,Triángulo de Amor Bizarro,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29/12/2023,
1,alternativa,Cala Vento,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/,01/12/2023,
2,alternativa,Joe Crepúsculo,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/,27/01/2024,
3,alternativa,Alice Wonder,1 Evento,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/,17/11/2023,
4,alternativa,Momentos Alhambra: Rikas,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...,14/12/2023,


In [109]:
len(events)

176

- Quiero duplicar las filas en el caso de que las fechas sean múltiples, ya que me servirá para la creacion de mi base de datos posterior y para su estudio comparado. 

- Lo que hago es duplicar aquellas filas que tienen varias fechas, poner este mismo evento en la fila consecutiva, debajo de la original, pero con la fecha de la 'fecha2'. 
- Posteriormente, crear un nuevo DataFrame con las filas duplicadas

In [110]:
nuevas_filas = []

# Iterar a través de las filas

for _, fila in events.iterrows():
    nuevas_filas.append(fila)
    if fila['fecha2']:
        nueva_fila = fila.copy()
        nueva_fila['fecha1'] = fila['fecha2']
        nueva_fila['fecha2'] = ''
        nuevas_filas.append(nueva_fila)

# Crear un nuevo DataFrame con las filas duplicadas
df_resultante = pd.DataFrame(nuevas_filas, columns = events.columns)

In [111]:
df_resultante.head()

,estilo,artista,eventos,precios,enlaces,fecha1,fecha2
0,alternativa,Triángulo de Amor Bizarro,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29/12/2023,
1,alternativa,Cala Vento,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/,01/12/2023,
2,alternativa,Joe Crepúsculo,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/,27/01/2024,
3,alternativa,Alice Wonder,1 Evento,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/,17/11/2023,
4,alternativa,Momentos Alhambra: Rikas,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...,14/12/2023,


In [112]:
len(df_resultante)

200

- A mi nuevo DataFrame le cambio el nombre, y así hago una copia del DataFrame. 

La limpieza de la primera parte de mi trabajo finaliza con limpiar el DataFrame definitivo, en cuanto al orden de columnas y utilidad de estas. 
- Elimino la columna de 'fecha2'
- Renombro la 'fecha1' a 'fechas'.
- Cambio la columna de 'fechas' a datetime, y divido esta columna en 3: 'dia', 'mes', 'año', por lo que se elimina la columna 'fechas'.
- Cambio el orden de las columnas.

Y YA TENGO MI TABLA DE ESTUDIO.

In [2]:
df_events = df_resultante.copy()

NameError: name 'df_resultante' is not defined

In [114]:
df_events = df_events.drop('fecha2', axis=1)

In [115]:
df_events = df_events.rename(columns={'fecha1': 'fechas'})

In [116]:
df_events.head()

,estilo,artista,eventos,precios,enlaces,fechas
0,alternativa,Triángulo de Amor Bizarro,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29/12/2023
1,alternativa,Cala Vento,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/,01/12/2023
2,alternativa,Joe Crepúsculo,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/,27/01/2024
3,alternativa,Alice Wonder,1 Evento,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/,17/11/2023
4,alternativa,Momentos Alhambra: Rikas,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...,14/12/2023


In [117]:
df_events = df_events[['estilo','artista', 'eventos', 'fechas', 'precios', 'enlaces']]

In [118]:
df_events.head()

,estilo,artista,eventos,fechas,precios,enlaces
0,alternativa,Triángulo de Amor Bizarro,1 Evento,29/12/2023,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
1,alternativa,Cala Vento,1 Evento,01/12/2023,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
2,alternativa,Joe Crepúsculo,1 Evento,27/01/2024,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/
3,alternativa,Alice Wonder,1 Evento,17/11/2023,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/
4,alternativa,Momentos Alhambra: Rikas,1 Evento,14/12/2023,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...


In [119]:
df_events['fechas'] = pd.to_datetime(df_events['fechas'], format='%d/%m/%Y', errors='coerce')

In [120]:
# Separar las fechas en columnas de día, mes y año

df_events['dia'] = df_events['fechas'].dt.day
df_events['mes'] = df_events['fechas'].dt.month
df_events['año'] = df_events['fechas'].dt.year

In [121]:
df_events.head()

,estilo,artista,eventos,fechas,precios,enlaces,dia,mes,año
0,alternativa,Triángulo de Amor Bizarro,1 Evento,2023-12-29,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29,12,2023
1,alternativa,Cala Vento,1 Evento,2023-12-01,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/,1,12,2023
2,alternativa,Joe Crepúsculo,1 Evento,2024-01-27,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/,27,1,2024
3,alternativa,Alice Wonder,1 Evento,2023-11-17,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/,17,11,2023
4,alternativa,Momentos Alhambra: Rikas,1 Evento,2023-12-14,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...,14,12,2023


In [122]:
df_events.drop('fechas', axis=1, inplace=True)

In [123]:
column_order = ['estilo', 'artista', 'eventos', 'dia', 'mes', 'año', 'precios', 'enlaces']
df_events = df_events[column_order]

In [124]:
df_events.head()

,estilo,artista,eventos,dia,mes,año,precios,enlaces
0,alternativa,Triángulo de Amor Bizarro,1 Evento,29,12,2023,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
1,alternativa,Cala Vento,1 Evento,1,12,2023,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
2,alternativa,Joe Crepúsculo,1 Evento,27,1,2024,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/
3,alternativa,Alice Wonder,1 Evento,17,11,2023,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/
4,alternativa,Momentos Alhambra: Rikas,1 Evento,14,12,2023,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...


In [125]:
df_events.tail()

,estilo,artista,eventos,dia,mes,año,precios,enlaces
171,clasica,Gala de Zarzuela en Navidad,1 Evento,17,12,2023,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...
172,clasica,Ton Koopman y Orquesta Barroca de Ámsterdam,1 Evento,29,4,2024,"desde 22,00 €",https://www.entradas.com/artist/ton-koopman-i-...
173,clasica,Hilary Hahn y Gianandrea Noseda,1 Evento,16,2,2024,"desde 32,00 €",https://www.entradas.com/artist/hilary-hahn-i-...
174,clasica,Hélène Grimaud,1 Evento,27,5,2024,"desde 22,00 €",https://www.entradas.com/artist/helene-grimaud/
175,clasica,Concierto Año Nuevo - The Orbist Internacional...,1 Evento,1,1,2024,"desde 35,00 €",https://www.entradas.com/artist/the-orbist-int...


In [140]:
df_events.columns

Index(['estilo', 'artista', 'eventos', 'dia', 'mes', 'año', 'precios',
       'enlaces'],
      dtype='object')

In [142]:
meses_a_trimestre = {1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2, 7: 3, 8: 3, 9: 3, 10: 4, 11: 4, 12: 4}

# Crear una nueva columna 'trimestre' basada en el mapeo de meses
df_events['trimestre'] = df_events['mes'].map(meses_a_trimestre)

# Mostrar el DataFrame después de agregar la nueva columna 'trimestre'
df_events.head()

,estilo,artista,eventos,dia,mes,año,precios,enlaces,trimestre
0,alternativa,Triángulo de Amor Bizarro,1 Evento,29,12,2023,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,4
1,alternativa,Cala Vento,1 Evento,1,12,2023,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/,4
2,alternativa,Joe Crepúsculo,1 Evento,27,1,2024,"desde 15,00 €",https://www.entradas.com/artist/joe-crepusculo/,1
3,alternativa,Alice Wonder,1 Evento,17,11,2023,"desde 27,00 €",https://www.entradas.com/artist/alice-wonder/,4
4,alternativa,Momentos Alhambra: Rikas,1 Evento,14,12,2023,"desde 15,00 €",https://www.entradas.com/artist/momentos-alham...,4


In [143]:
column_order = ['estilo', 'artista', 'eventos', 'dia', 'mes', 'año', 'trimestre', 'precios', 'enlaces']
df_events = df_events[column_order]

In [146]:
df_events.tail()

,estilo,artista,eventos,dia,mes,año,trimestre,precios,enlaces
171,clasica,Gala de Zarzuela en Navidad,1 Evento,17,12,2023,4,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...
172,clasica,Ton Koopman y Orquesta Barroca de Ámsterdam,1 Evento,29,4,2024,2,"desde 22,00 €",https://www.entradas.com/artist/ton-koopman-i-...
173,clasica,Hilary Hahn y Gianandrea Noseda,1 Evento,16,2,2024,1,"desde 32,00 €",https://www.entradas.com/artist/hilary-hahn-i-...
174,clasica,Hélène Grimaud,1 Evento,27,5,2024,2,"desde 22,00 €",https://www.entradas.com/artist/helene-grimaud/
175,clasica,Concierto Año Nuevo - The Orbist Internacional...,1 Evento,1,1,2024,1,"desde 35,00 €",https://www.entradas.com/artist/the-orbist-int...


In [25]:
import pandas as pd

In [36]:
data = pd.read_csv('../Data/Clean_data/events_estilo.csv', encoding='latin1')
data.head()

,estilo,artista,eventos,dia,mes,aÃ±o,trimestre,precios,enlaces
0,alternativa,TriÃ¡ngulo de Amor Bizarro,1 Evento,29,12,2023,4,"desde 19,80 â¬",https://www.entradas.com/artist/triangulo-de-a...
1,alternativa,Cala Vento,1 Evento,1,12,2023,4,"desde 19,80 â¬",https://www.entradas.com/artist/cala-vento/
2,alternativa,Joe CrepÃºsculo,1 Evento,27,1,2024,1,"desde 15,00 â¬",https://www.entradas.com/artist/joe-crepusculo/
3,alternativa,Alice Wonder,1 Evento,17,11,2023,4,"desde 27,00 â¬",https://www.entradas.com/artist/alice-wonder/
4,alternativa,Momentos Alhambra: Rikas,1 Evento,14,12,2023,4,"desde 15,00 â¬",https://www.entradas.com/artist/momentos-alham...


In [37]:
len(data)

200

In [38]:

# Aplicar la función str.replace para quitar "desde" y convertir a tipo numérico
data['precios'] = data['precios'].str.replace('desde', '').str.replace('â\x82¬', '').str.replace(',', '.').str.strip()

data

,estilo,artista,eventos,dia,mes,aÃ±o,trimestre,precios,enlaces
0,alternativa,TriÃ¡ngulo de Amor Bizarro,1 Evento,29,12,2023,4,19.80,https://www.entradas.com/artist/triangulo-de-a...
1,alternativa,Cala Vento,1 Evento,1,12,2023,4,19.80,https://www.entradas.com/artist/cala-vento/
2,alternativa,Joe CrepÃºsculo,1 Evento,27,1,2024,1,15.00,https://www.entradas.com/artist/joe-crepusculo/
3,alternativa,Alice Wonder,1 Evento,17,11,2023,4,27.00,https://www.entradas.com/artist/alice-wonder/
4,alternativa,Momentos Alhambra: Rikas,1 Evento,14,12,2023,4,15.00,https://www.entradas.com/artist/momentos-alham...
...,...,...,...,...,...,...,...,...,...
195,clasica,Gala de Zarzuela en Navidad,1 Evento,17,12,2023,4,39.00,https://www.entradas.com/artist/gala-de-zarzue...
196,clasica,Ton Koopman y Orquesta Barroca de Ãmsterdam,1 Evento,29,4,2024,2,22.00,https://www.entradas.com/artist/ton-koopman-i-...
197,clasica,Hilary Hahn y Gianandrea Noseda,1 Evento,16,2,2024,1,32.00,https://www.entradas.com/artist/hilary-hahn-i-...
198,clasica,HÃ©lÃ¨ne Grimaud,1 Evento,27,5,2024,2,22.00,https://www.entradas.com/artist/helene-grimaud/


In [39]:
data['precios'] = pd.to_numeric(data['precios'], errors='coerce').round(2)

data.head()

,estilo,artista,eventos,dia,mes,aÃ±o,trimestre,precios,enlaces
0,alternativa,TriÃ¡ngulo de Amor Bizarro,1 Evento,29,12,2023,4,19.8,https://www.entradas.com/artist/triangulo-de-a...
1,alternativa,Cala Vento,1 Evento,1,12,2023,4,19.8,https://www.entradas.com/artist/cala-vento/
2,alternativa,Joe CrepÃºsculo,1 Evento,27,1,2024,1,15.0,https://www.entradas.com/artist/joe-crepusculo/
3,alternativa,Alice Wonder,1 Evento,17,11,2023,4,27.0,https://www.entradas.com/artist/alice-wonder/
4,alternativa,Momentos Alhambra: Rikas,1 Evento,14,12,2023,4,15.0,https://www.entradas.com/artist/momentos-alham...


In [147]:
driver.quit()

In [40]:
data.to_csv('events_estilo.csv', index=False)